In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
train_dir = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\training'
val_dir = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\validation'
test_dir = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\test'

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 증식을 위한 ImageDataGenerator 초기화
train_datagen = ImageDataGenerator(
    rescale=1./255,  # 이미지 스케일 조정
    #width_shift_range=0.2,  # 이미지를 수평으로 랜덤 이동 시키는 범위
    #height_shift_range=0.2,  # 이미지를 수직으로 랜덤 이동 시키는 범위
    #zoom_range=0.2,  # 랜덤하게 확대할 범위
    #horizontal_flip=True,  # 수평 방향으로 이미지 뒤집기
    #fill_mode='nearest'  # 회전이나 너비/높이 이동으로 인해 새롭게 생성해야 할 픽셀을 채우는 방법
)

# 검증과 테스트 데이터셋에는 데이터 증식을 적용하지 않음
test_datagen = ImageDataGenerator(rescale=1./255)

# 훈련 데이터셋을 위한 제너레이터
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

# 검증 데이터셋을 위한 제너레이터
validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)

# 테스트 데이터셋을 위한 제너레이터
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)


Found 3940 images belonging to 4 classes.
Found 846 images belonging to 4 classes.
Found 844 images belonging to 4 classes.


In [2]:
def build_model(hp):
    base_model = tf.keras.applications.resnet_v2.ResNet50V2(weights="imagenet",
                                                         include_top=False, 
                                                         input_shape=(128, 128, 3))
    for layer in base_model.layers:
        base_model.trainable = False
    for layer in base_model.layers[-9:]:
        base_model.trainable = True
    

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    # Dropout 적용
    model.add(layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)))

    # L2 규제를 적용한 Dense 레이어 추가
    model.add(layers.Dense(units=hp.Int('units', min_value=128, max_value=512, step=128),
                           activation='relu',
                           kernel_regularizer=regularizers.l2(hp.Float('l2', min_value=1e-5, max_value=1e-2, sampling='LOG'))))
    
    # Dense 레이어의 유닛 수를 하이퍼파라미터로 사용
    #model.add(layers.Dense(units=hp.Int('units', min_value=128, max_value=512, step=128), activation = 'relu'))
    model.add(layers.Dense(4, activation = 'softmax'))
    
    # learning_rate를 하이퍼파라미터로 사용
    model.compile(optimizer=optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-6, max_value=1e-4, sampling='LOG')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [3]:
import keras_tuner as kt

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,  # 시도할 하이퍼파라미터 조합의 최대 개수
    executions_per_trial=1,  # 각 하이퍼파라미터 설정을 평가하기 위해 모델을 훈련시킬 횟수
    directory='E:\AImodel\ResNet_RandomSearch',  # 튜닝 세션의 결과를 저장할 디렉토리 이름
    project_name='ResNet-BicycleCrunch-02',) # 프로젝트 이름

tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
learning_rate (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.0001, 'step': None, 'sampling': 'log'}


In [4]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

tuner.search(train_generator,
             steps_per_epoch=train_generator.samples // train_generator.batch_size,
             validation_data=validation_generator, 
             validation_steps=validation_generator.samples // validation_generator.batch_size,
             epochs=25,
             callbacks=[early_stopping])


# 최적의 하이퍼파라미터 가져오기
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# 최적의 하이퍼파라미터로 모델 빌드
model = tuner.hypermodel.build(best_hps)

Trial 20 Complete [00h 01m 32s]
val_accuracy: 0.5949519276618958

Best val_accuracy So Far: 0.692307710647583
Total elapsed time: 00h 34m 37s


In [5]:
# 모든 최적 하이퍼파라미터 출력
for hp in best_hps.values:
    print(f"The optimal value for {hp} is {best_hps.get(hp)}")

# 모델 훈련
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs=25,
                    callbacks=[early_stopping])

# 모델 평가 (테스트 데이터셋)
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('\n테스트 정확도:', test_acc)

The optimal value for units is 384
The optimal value for learning_rate is 8.532036969883326e-05
Epoch 1/25
123/123 [==============================] - 12s 75ms/step - loss: 1.4463 - accuracy: 0.3736 - val_loss: 1.5373 - val_accuracy: 0.2812
Epoch 2/25
123/123 [==============================] - 9s 71ms/step - loss: 1.0492 - accuracy: 0.5591 - val_loss: 1.3529 - val_accuracy: 0.3966
Epoch 3/25
123/123 [==============================] - 9s 71ms/step - loss: 0.8236 - accuracy: 0.6720 - val_loss: 1.3014 - val_accuracy: 0.4856
Epoch 4/25
123/123 [==============================] - 9s 71ms/step - loss: 0.6650 - accuracy: 0.7398 - val_loss: 1.0673 - val_accuracy: 0.5901
Epoch 5/25
123/123 [==============================] - 9s 71ms/step - loss: 0.4766 - accuracy: 0.8227 - val_loss: 1.0768 - val_accuracy: 0.6310
Epoch 6/25
123/123 [==============================] - 9s 71ms/step - loss: 0.3616 - accuracy: 0.8705 - val_loss: 1.1265 - val_accuracy: 0.6238
Epoch 7/25
123/123 [=========================